In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd
import plotly.express as px

**1. Carga y Exploración de Datos**

In [2]:
# Usamos la biblioteca google para poder usar archivos en nuestro drive.
from google.colab import drive
# Este comando conecta colab con drive.
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/MyDrive/Skillnest/ML/CORES/vehicles_sample.csv"
df = pd.read_csv(path)

**2. LIMPIEZA Y PREPROCESAMIENTO**

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            10000 non-null  int64  
 1   url           10000 non-null  object 
 2   region        10000 non-null  object 
 3   region_url    10000 non-null  object 
 4   price         10000 non-null  int64  
 5   year          9974 non-null   float64
 6   manufacturer  9611 non-null   object 
 7   model         9888 non-null   object 
 8   condition     5958 non-null   object 
 9   cylinders     5824 non-null   object 
 10  fuel          9916 non-null   object 
 11  odometer      9896 non-null   float64
 12  title_status  9788 non-null   object 
 13  transmission  9942 non-null   object 
 14  VIN           6205 non-null   object 
 15  drive         6990 non-null   object 
 16  size          2805 non-null   object 
 17  type          7884 non-null   object 
 18  paint_color   7010 non-null

In [6]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7315883828,https://lakeland.craigslist.org/ctd/d/lakeland...,lakeland,https://lakeland.craigslist.org,36990,2017.0,ford,f150 super cab lariat,good,6 cylinders,...,NaN,pickup,white,https://images.craigslist.org/00s0s_lRS7etJoVE...,Carvana is the safer way to buy a car During t...,NaN,fl,28.0400,-81.9600,2021-05-02T15:31:06-0400
1,7314599643,https://quadcities.craigslist.org/ctd/d/waterl...,"quad cities, IA/IL",https://quadcities.craigslist.org,27995,2006.0,chevrolet,corvette,good,8 cylinders,...,NaN,convertible,black,https://images.craigslist.org/00101_aa4DyXpKu0...,2006 *** Chevrolet Corvette Convertible Conver...,NaN,il,42.4778,-92.3661,2021-04-29T18:46:35-0500
2,7308399808,https://littlerock.craigslist.org/ctd/d/clinto...,little rock,https://littlerock.craigslist.org,78423,2015.0,chevrolet,corvette,NaN,8 cylinders,...,NaN,convertible,NaN,https://images.craigslist.org/00A0A_kJsL7mVMCg...,➔ Want to see more pictures?Paste this link to...,NaN,ar,38.4018,-93.7850,2021-04-17T14:01:33-0500
3,7312663807,https://wheeling.craigslist.org/ctd/d/follansb...,northern panhandle,https://wheeling.craigslist.org,14000,2013.0,bmw,328i,NaN,NaN,...,NaN,NaN,NaN,https://images.craigslist.org/00K0K_2oCjTKrjd9...,"**Deals, Deals, Deals** Beautiful 2013 BMW 3-S...",NaN,oh,40.3203,-80.6250,2021-04-25T23:53:42-0400
4,7315368523,https://eugene.craigslist.org/ctd/d/cottage-gr...,eugene,https://eugene.craigslist.org,676,2019.0,chevrolet,suburban ls,NaN,8 cylinders,...,NaN,NaN,black,https://images.craigslist.org/00H0H_3hFsa4lTxO...,2019 Chevrolet Suburban LS Brads Chevy - ☎️ ...,NaN,or,43.7839,-123.0529,2021-05-01T10:04:24-0700


In [7]:
# Ver cuántos ids únicos hay
print("IDs únicos:", df["id"].nunique())
print("Total de filas:", len(df))

IDs únicos: 10000
Total de filas: 10000


No hay duplicados por id, mas adelante borraria esta columna

In [8]:
# Porcentaje de nulos por columna
porcentaje_nulos = df.isna().mean().sort_values(ascending=False) * 100

# Mostrar en forma de tabla
porcentaje_nulos = porcentaje_nulos.round(2).reset_index()
porcentaje_nulos.columns = ["Columna", "Porcentaje de Nulos"]
porcentaje_nulos

,Columna,Porcentaje de Nulos
0,county,100.00
1,size,71.95
2,cylinders,41.76
3,condition,40.42
4,VIN,37.95
5,drive,30.10
6,paint_color,29.90
7,type,21.16
8,manufacturer,3.89
9,title_status,2.12


In [9]:
df["url"]

,url
0,https://lakeland.craigslist.org/ctd/d/lakeland...
1,https://quadcities.craigslist.org/ctd/d/waterl...
2,https://littlerock.craigslist.org/ctd/d/clinto...
3,https://wheeling.craigslist.org/ctd/d/follansb...
4,https://eugene.craigslist.org/ctd/d/cottage-gr...
...,...
9995,https://baltimore.craigslist.org/ctd/d/baltimo...
9996,https://humboldt.craigslist.org/cto/d/cutten-9...
9997,https://saginaw.craigslist.org/cto/d/saginaw-2...
9998,https://newhaven.craigslist.org/ctd/d/new-have...


In [10]:
df["region"]

,region
0,lakeland
1,"quad cities, IA/IL"
2,little rock
3,northern panhandle
4,eugene
...,...
9995,baltimore
9996,humboldt county
9997,saginaw-midland-baycity
9998,new haven


In [11]:
df["region_url"]

,region_url
0,https://lakeland.craigslist.org
1,https://quadcities.craigslist.org
2,https://littlerock.craigslist.org
3,https://wheeling.craigslist.org
4,https://eugene.craigslist.org
...,...
9995,https://baltimore.craigslist.org
9996,https://humboldt.craigslist.org
9997,https://saginaw.craigslist.org
9998,https://newhaven.craigslist.org


In [12]:
df["VIN"]

,VIN
0,1FTFX1EG9HKD14814
1,NaN
2,NaN
3,NaN
4,1GNSKGKC7KR124145
...,...
9995,KNAFZ6A39G5606001
9996,NaN
9997,NaN
9998,2C3CDZBT2HH666317


In [13]:
df["size"]

,size
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
9995,NaN
9996,full-size
9997,NaN
9998,NaN


In [14]:
df["type"]

,type
0,pickup
1,convertible
2,convertible
3,NaN
4,NaN
...,...
9995,coupe
9996,NaN
9997,sedan
9998,coupe


In [15]:
df["image_url"]

,image_url
0,https://images.craigslist.org/00s0s_lRS7etJoVE...
1,https://images.craigslist.org/00101_aa4DyXpKu0...
2,https://images.craigslist.org/00A0A_kJsL7mVMCg...
3,https://images.craigslist.org/00K0K_2oCjTKrjd9...
4,https://images.craigslist.org/00H0H_3hFsa4lTxO...
...,...
9995,https://images.craigslist.org/01414_3FlFAF3lVp...
9996,https://images.craigslist.org/00M0M_bIJt4cEQnf...
9997,https://images.craigslist.org/00808_8YBW6bXSat...
9998,https://images.craigslist.org/00Y0Y_4YoVaGms0N...


In [16]:
df["description"]

,description
0,Carvana is the safer way to buy a car During t...
1,2006 *** Chevrolet Corvette Convertible Conver...
2,➔ Want to see more pictures?Paste this link to...
3,"**Deals, Deals, Deals** Beautiful 2013 BMW 3-S..."
4,2019 Chevrolet Suburban LS Brads Chevy - ☎️ ...
...,...
9995,Carvana is the safer way to buy a car During t...
9996,This is a 1999 Ford Club Wagon passenger Van t...
9997,2008 BUICK LUCERNE CX 3.8L V6 POWER WINDOWS ...
9998,Carvana is the safer way to buy a car During t...


In [17]:
df["lat"]

,lat
0,28.040000
1,42.477800
2,38.401800
3,40.320300
4,43.783900
...,...
9995,39.300000
9996,40.759200
9997,43.424800
9998,41.310000


In [18]:
df["long"]

,long
0,-81.960000
1,-92.366100
2,-93.785000
3,-80.625000
4,-123.052900
...,...
9995,-76.610000
9996,-124.159300
9997,-83.974500
9998,-72.920000


In [19]:
df["posting_date"]

,posting_date
0,2021-05-02T15:31:06-0400
1,2021-04-29T18:46:35-0500
2,2021-04-17T14:01:33-0500
3,2021-04-25T23:53:42-0400
4,2021-05-01T10:04:24-0700
...,...
9995,2021-05-03T13:51:19-0400
9996,2021-04-28T11:11:26-0700
9997,2021-04-30T15:18:04-0400
9998,2021-04-24T11:51:12-0400


Analizando las varibles anteriores de tipo texto y algunas númericas, además del porcentaje de valores nulos que poseen, procedo a eliminar las variables:

id, url, region_url, VIN, image_url y description, porque las considero pocos utiles en el analisis posterior, algunas son textos y otras son tipo codigos.

Tambien eliminare county y size, ya que poseen un gran porcentaje de valores nulos respecto al total, 100% y 71.95% respectivamente.

Lat y long representan la ubicacion, ademas a ambas les faltan los mismos valores, no las considero esenciales para el analisis porque son coordenadas exactas y para la ubicacion cuento con region y state.

De igual forma eliminare posting_date ya que contiene fecha y hora, y no es relevante en el analisis posterior.

In [20]:
# Eliminacion de columnas irrelevantes para el analisis, y con gran porcentaje de valores nulos.
columnas_a_eliminar = ["id", "url", "region_url", "VIN", "image_url", "description", "county", "size","lat","long","posting_date"]

df.drop(columns=columnas_a_eliminar, inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   region        10000 non-null  object 
 1   price         10000 non-null  int64  
 2   year          9974 non-null   float64
 3   manufacturer  9611 non-null   object 
 4   model         9888 non-null   object 
 5   condition     5958 non-null   object 
 6   cylinders     5824 non-null   object 
 7   fuel          9916 non-null   object 
 8   odometer      9896 non-null   float64
 9   title_status  9788 non-null   object 
 10  transmission  9942 non-null   object 
 11  drive         6990 non-null   object 
 12  type          7884 non-null   object 
 13  paint_color   7010 non-null   object 
 14  state         10000 non-null  object 
dtypes: float64(2), int64(1), object(12)
memory usage: 1.1+ MB


In [22]:
# Analizar estadisticas basicas
df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
price,10000.0,17968.95,24946.87,0.0,5995.00,13987.5,26590.0,1234567.0
year,9974.0,2011.35,9.18,1903.0,2008.00,2013.0,2017.0,2022.0
odometer,9896.0,101767.33,279643.86,0.0,37877.25,86444.0,135000.0,10000000.0


Analizando las estadisticas basicas:

En price el maximo es muy elevado, mas adelante lo analizare, posible outlier. En year parece estar todo bien, año 1900 podrian ser autos clasicos, ademas los valores faltantes que hay los reemplazare por la mediana que es 2013 lo cual es mas representativa que la media. En odometer hay valores extremos igual, posibles outliers, mas adelante los analizo.

In [23]:
# Ver los 10 valores extremos en price
df[["price"]].sort_values(by="price", ascending=False).head(10)

,price
300,1234567
153,1111111
3435,1000000
6034,299500
531,209999
766,199999
6735,197999
6918,179988
4592,145000
4722,135000


In [24]:
# Ver los 10 valores extremos en odometer
df[["odometer"]].sort_values(by="odometer", ascending=False).head(10)

,odometer
8542,10000000.0
6737,10000000.0
6729,9999999.0
1398,9999999.0
4663,9999999.0
4520,9999999.0
6253,7777777.0
4375,7654321.0
7910,2300002.0
319,2120691.0


Luego de analizar que existen valores muy altos para price y odometer, decido eliminar esos valores atipicos segun el percentil 99

Los valores más allá del percentil 99 suelen ser outliers o errores, los cuales pueden distorsionar el análisis o el entrenamiento del modelo, lo que se hara es, filtrar solo el 1% más alto, entonces se eliminan pocos datos, por ende no se pierde mucha información.

In [25]:
# Eliminar valores otuliers para price y odometer

# Calcular percentil 99 para price y odometer
umbral_precio = df["price"].quantile(0.99)
umbral_odometer = df["odometer"].quantile(0.99)

# Filtrar para eliminar outliers en price y odometer
df_limpio = df[(df["price"] <= umbral_precio) & (df["odometer"] <= umbral_odometer)]

In [26]:
# Reemplazar por moda lo valores nulos en year y odometer
df_limpio.loc[:, "year"] = df_limpio["year"].fillna(df_limpio["year"].median())
df_limpio.loc[:, "odometer"] = df_limpio["odometer"].fillna(df_limpio["odometer"].median())

In [27]:
# Imputar columnas cateogircas con la moda
columnas_categoricas_con_nulos = ["manufacturer", "model", "condition", "cylinders", "fuel","title_status", "transmission", "drive", "type", "paint_color"]

for col in columnas_categoricas_con_nulos:
    moda = df_limpio[col].mode()[0]
    df_limpio.loc[:, col] = df_limpio[col].fillna(moda)

In [28]:
# Verificacion final nulos
df_limpio.isna().sum()

,0
region,0
price,0
year,0
manufacturer,0
model,0
condition,0
cylinders,0
fuel,0
odometer,0
title_status,0


In [29]:
# Verificando
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9701 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   region        9701 non-null   object 
 1   price         9701 non-null   int64  
 2   year          9701 non-null   float64
 3   manufacturer  9701 non-null   object 
 4   model         9701 non-null   object 
 5   condition     9701 non-null   object 
 6   cylinders     9701 non-null   object 
 7   fuel          9701 non-null   object 
 8   odometer      9701 non-null   float64
 9   title_status  9701 non-null   object 
 10  transmission  9701 non-null   object 
 11  drive         9701 non-null   object 
 12  type          9701 non-null   object 
 13  paint_color   9701 non-null   object 
 14  state         9701 non-null   object 
dtypes: float64(2), int64(1), object(12)
memory usage: 1.2+ MB


**3. EXPLORACION DE DATOS**

In [30]:
df_limpio.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
price,9701.0,17053.34,13957.06,0.0,5995.0,13950.0,25995.0,68777.0
year,9701.0,2011.46,8.89,1927.0,2009.0,2013.0,2017.0,2022.0
odometer,9701.0,90684.13,60610.22,0.0,38269.0,86320.0,134000.0,285000.0


In [31]:
# Pie chart para distribución de fuel

fig = px.pie(
    df_limpio,
    names="fuel",
    title="Distribución de vehículos por tipo de combustible",
    hole=0.3  # gráfico tipo donut
)
fig.update_traces(textinfo="percent+label")
fig.update_layout(title_font_size=18)
fig.show()

El gráfico muestra la proporción de vehículos según el tipo de combustible utilizado. Se observa que:

Existe un tipo predominante de combustible que es tipo gas con un 85.1%, lo que sugiere que la mayoría de los vehículos ofertados comparten esta característica.

Los combustibles como eléctrico, híbrido o diésel, tienen una participación minoritaria.

In [32]:
# Histograma para variable numérica price

fig = px.histogram(
    df_limpio,
    x="price",
    nbins=30,
    title="Distribución de precios de vehículos usados"
)
fig.update_layout(bargap=0.1, title_font_size=18)
fig.show()

El histograma muestra cómo se distribuyen los precios de los vehículos en el dataset, la mayoría de los vehículos se concentran en el rango de precios bajos a medios, entre aproximadamente 0 y 30k.

A medida que el precio aumenta, la frecuencia disminuye drásticamente, lo que indica que los vehículos más caros son menos comunes.

In [33]:
# Histograma agrupado para precios de vehículos por combustible y tipo de transmisión

fig = px.histogram(
    df_limpio,
    x="fuel",
    y="price",
    color="transmission",
    barmode="group",
    title="Distribución de precios de vehículos usados según tipo de combustible y transmisión"
)
fig.update_layout(title_font_size=18)
fig.show()

Este gráfico muestra cómo varía el precio promedio de los vehículos según el tipo de combustible (fuel) y la transmisión (transmission):

En general, los vehículos con transmisión automática tienden a tener precios más altos en casi todos los tipos de combustible.

El tipo de combustible también influye: por ejemplo, algunos combustibles como eléctrico o híbrido presentan menos datos, posiblemente por su tecnología y menor disponibilidad.

Se pueden observar diferencias claras entre subgrupos: por ejemplo, dentro del combustible "gasolina", la transmisión automática supera consistentemente en precio a la manual.

**4. MODELADO Y EVALUACION**

Entrenar y evaluar Linear Regression y Random Forest

In [34]:
df_limpio["condition"].unique()

array(['good', 'excellent', 'like new', 'fair', 'new', 'salvage'],
      dtype=object)

In [35]:
df_limpio["cylinders"].unique()

array(['6 cylinders', '8 cylinders', '4 cylinders', 'other',
       '5 cylinders', '10 cylinders', '3 cylinders', '12 cylinders'],
      dtype=object)

variables categoricas:

nominales: "manufacturer", "model", "fuel", "title_status", "transmission", "drive", "type", "paint_color", "state".

Ya que no tienen un orden lógico.

ordinales: "condition" y "cylinders".

condition: representa el estado del vehículo, ademas tiene una progresión de calidad que sigue una escala:
"salvage" < "fair" < "good" < "excellent" < "like new" < "new".
Estas categorías implican una mejora progresiva en la condición del vehículo, y por lo tanto, el orden importa.

cylinders: Se refiere a la cantidad de cilindros del motor. También tiene un orden natural:
"3 cylinders" < "4 cylinders" < "5 cylinders" < "6 cylinders" < "8 cylinders" < "10 cylinders" < "12 cylinders" < "other".
A mayor cantidad de cilindros, podria ser más potente es el motor y más caro el auto.

varibales numericas:

numericas: "year" y "odometer".
Ambas son númericas continuas, el año es un valor temporal medible, mientras que odometer, es un número que representa cuántos kilometros o millas ha recorrido el auto.

In [36]:
# Definir features y target
X = df_limpio.drop(columns="price")
y = df_limpio["price"]

In [37]:
# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
# Columnas numéricas, ordinales y nominales
num_cols = ["year", "odometer"]
ord_cols = ["condition", "cylinders"]
nom_cols = ["manufacturer", "model", "fuel", "title_status", "transmission", "drive", "type", "paint_color", "state"]

In [39]:
# Pipeline para columnas numéricas: solo escalar.
num_pipeline = Pipeline(steps=[
    ("scaler", StandardScaler())
])

# Pipeline para columnas ordinales: codificar y escalar.
ord_pipeline = Pipeline(steps=[
    ("ordinal", OrdinalEncoder(categories=[
        ["salvage", "fair", "good", "excellent", "like new", "new"],  # condition
        ["3 cylinders", "4 cylinders", "5 cylinders", "6 cylinders",
         "8 cylinders", "10 cylinders", "12 cylinders", "other"]     # cylinders
    ])),
    ("scaler", StandardScaler())
])

# Pipeline para columnas nominales: one-hot encoding
nom_pipeline = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

Modelo 1: Regresión Lineal (LinearRegression)

In [40]:
# ColumnTransformer: combina todos los preprocesamientos
preprocessor_lr = ColumnTransformer(transformers=[
    ("num", num_pipeline, num_cols),
    ("ord", ord_pipeline, ord_cols),
    ("nom", nom_pipeline, nom_cols)
])

In [41]:
# Pipeline para Regresión Lineal
pipeline_lr = Pipeline(steps=[
    ("preprocessing", preprocessor_lr),
    ("regressor", LinearRegression())
])

In [42]:
# Entrenar el modelo
pipeline_lr.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'odometer']),
                                                 ('ord',
                                                  Pipeline(steps=[('ordinal',
                                                                   OrdinalEncoder(categories=[['salvage',
                                                                                               'fair',
                                                                                               'good',
                                                                                               'excellent',
                                                                                               'like '
                                                                                               'new',
                                                                                               'new'],
                                                                                              ['3 '
                                                                                               'cylinders',
                                                                                               '4 '
                                                                                               'cylinders',
                                                                                               '5 '
                                                                                               'cylinders',
                                                                                               '6 '
                                                                                               'cylinders',
                                                                                               '8 '
                                                                                               'cylinders',
                                                                                               '10 '
                                                                                               'cylinders',
                                                                                               '12 '
                                                                                               'cylinders',
                                                                                               'other']])),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['condition', 'cylinders']),
                                                 ('nom',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['manufacturer', 'model',
                                                   'fuel', 'title_status',
                                                   'transmission', 'drive',
                                                   'type', 'paint_color',
                                                   'state'])])),
                ('regressor', LinearRegression())])

In [43]:
# Predecir
y_pred_lr = pipeline_lr.predict(X_test)

In [44]:
# Evaluar
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = mse_lr ** 0.5  # raíz cuadrada del MSE
r2_lr = r2_score(y_test, y_pred_lr)

Modelo 2: Bosque Aleatorio (RandomForestRegressor)

In [45]:
# ColumnTransformer: combina todos los preprocesamientos
preprocessor_rf = ColumnTransformer(transformers=[
    ("num", num_pipeline, num_cols),
    ("ord", ord_pipeline, ord_cols),
    ("nom", nom_pipeline, nom_cols)
])

In [62]:
# Pipeline para Random Forest
pipeline_rf = Pipeline(steps=[
    ("preprocessing", preprocessor_rf),
    ("model", RandomForestRegressor(n_estimators=50, random_state=42))
])

In [63]:
# Entrenar el modelo
pipeline_rf.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'odometer']),
                                                 ('ord',
                                                  Pipeline(steps=[('ordinal',
                                                                   OrdinalEncoder(categories=[['salvage',
                                                                                               'fair',
                                                                                               'good',
                                                                                               'excellent',
                                                                                               'like '
                                                                                               'new',
                                                                                               'new'],
                                                                                              ['3 '
                                                                                               'cylinders',
                                                                                               '4 '
                                                                                               'cylinders',
                                                                                               '5 '
                                                                                               'cylinders',
                                                                                               '6 '
                                                                                               'cylinders',
                                                                                               '8 '
                                                                                               'cylinders',
                                                                                               '10 '
                                                                                               'cylinders',
                                                                                               '12 '
                                                                                               'cylinders',
                                                                                               'other']])),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['condition', 'cylinders']),
                                                 ('nom',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['manufacturer', 'model',
                                                   'fuel', 'title_status',
                                                   'transmission', 'drive',
                                                   'type', 'paint_color',
                                                   'state'])])),
                ('model',
                 RandomForestRegressor(n_estimators=50, random_state=42))])

In [65]:
# Predecir
y_pred_rf = pipeline_rf.predict(X_test)

In [66]:
# Evaluar
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = mse_rf ** 0.5  # raíz cuadrada del MSE
r2_rf = r2_score(y_test, y_pred_rf)

In [67]:
# Mostrar resultados
print("Regresión Lineal")
print(f"  MSE:  {mse_lr:.2f}")
print(f"  RMSE: {rmse_lr:.2f}")
print(f"  R²:   {r2_lr:.4f}")

print("Random Forest Regressor")
print(f"  MSE:  {mse_rf:.2f}")
print(f"  RMSE: {rmse_rf:.2f}")
print(f"  R²:   {r2_rf:.4f}")

Regresión Lineal
  MSE:  108114622.16
  RMSE: 10397.82
  R²:   0.4554
Random Forest Regressor
  MSE:  85041130.62
  RMSE: 9221.77
  R²:   0.5716


Tras comparar los resultados de ambos modelos utilizando las métricas MSE, RMSE y R², se puede concluir que el Random Forest Regressor es el modelo con mejor desempeño, con valores de:

MSE:  85041130.62
RMSE: 9221.77
R²:   0.5716

El Random Forest logra un menor MSE y un mejor R².

Esto indica que explica mejor la variabilidad del precio y tiene predicciones más precisas en comparación con la regresión lineal.

**5. OPTIMIZACION DEL MODELO**

In [61]:
# Optimizacion de hiperparametros
forest_params = {
    "model__n_estimators": [100, 50]
}

forest_grid = GridSearchCV(pipeline_rf, forest_params, cv=3, scoring="r2")
forest_grid.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['year',
                                                                          'odometer']),
                                                                        ('ord',
                                                                         Pipeline(steps=[('ordinal',
                                                                                          OrdinalEncoder(categories=[['salvage',
                                                                                                                      'fair',
                                                                                                                      'good',
                                                                                                                      'excellent',
                                                                                                                      'like '
                                                                                                                      'new',
                                                                                                                      'new'],
                                                                                                                     ['3 '
                                                                                                                      'cylinders',
                                                                                                                      '4 '
                                                                                                                      'cylinders',
                                                                                                                      '5 '
                                                                                                                      'cylinders',
                                                                                                                      '6 '
                                                                                                                      'cylinders',
                                                                                                                      '8 '
                                                                                                                      'c...
                                                                                                                      'other']])),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['condition',
                                                                          'cylinders']),
                                                                        ('nom',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['manufacturer',
                                                                          'model',
                                                                          'fuel',
                                            

In [68]:
# Evaluación.
forest_best = forest_grid.best_estimator_
y_pred_forest = forest_best.predict(X_test)

print("Random Forest Regressor")
print("Mejores parámetros:", forest_grid.best_params_)
print("R²:", r2_score(y_test, y_pred_forest))

Random Forest Regressor
Mejores parámetros: {'model__n_estimators': 100}
R²: 0.5738137947885877


La búsqueda encontró que el modelo con 100 árboles (n_estimators = 100) ofrece el mejor rendimiento dentro de los valores evaluados.

Un R² de 0.5738 indica que el modelo optimizado es capaz de explicar aproximadamente el 57% de la variabilidad en los precios de los vehículos usados.

**CONCLUSIONES FINALES**

Se implementaron dos modelos de regresión para predecir el precio de vehículos usados:

Regresión Lineal: MSE:  108114622.16 RMSE: 10397.82 R²:   0.4554

Random Forest Regressor (sin optimizar):  MSE:  85041130.62 RMSE: 9221.77 R²:   0.5716

Random Forest Regressor (optimizado con GridSearchCV): Mejores parámetros: model__n_estimators': 100 y R²: 0.5738137947885877

El modelo Random Forest optimizado logró el mejor desempeño general con un R² de 0.5738, superando tanto a la regresión lineal como a la versión de random forest sin optimizar (R² de 0.5716).
La versión optimizada con n_estimators = 100 tuvo un rendimiento ligeramente mayor.

Aun así, ambas versiones de Random Forest superan a la regresión lineal, lo que confirma que un modelo no lineal se ajusta mejor a la complejidad del problema.

El mejor modelo para este problema es Random Forest Regressor optimizado, ya que entrega la mejor métrica R² y un menor RMSE. La regresión lineal, aunque más simple y rápida, no logra modelar adecuadamente la variabilidad del precio de los vehículos.

De los gráficos se interpreta que:

La variable fuel puede ser una característica relevante para predecir el precio, ya que ciertos tipos de combustible tienden a asociarse con vehículos más caros o más nuevos.

La varibale price presenta una distribución no normal y sesgada, lo que justifica el uso de modelos más robustos como Random Forest. Además, es clave haber eliminado los outliers para evitar que afecten la escala y el aprendizaje del modelo.

La relación entre fuel, transmission y price sugiere que estas variables podrían ser importantes en los modelos de regresión. Además, la interacción entre variables categóricas podría ser aprovechada por modelos no lineales como Random Forest.